# 初始化

In [1]:
import os
import sys
from pathlib import Path
import time
import json

# 添加项目根目录到Python路径
# sys.path.insert(0, str(Path(__file__).parent))

# 禁用缓存以提高性能
os.environ['DISABLE_LANGCHAIN_CACHE'] = 'true'

from core.react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel
from core.langchain_agent_tool import AgentToolWrapper, create_langchain_tool
from core.debug_tools import (
    check_and_compress_debug_notes, 
    create_debug_tools
)
from langchain_core.tools import tool

# 如果使用 Gemini 需要导入 httpx
try:
    import httpx
    HAS_HTTPX = True
except ImportError:
    HAS_HTTPX = False


# 清空目录

In [30]:
work_dir = "/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/output/blog"
# 清空 work_dir 目录下的所有文件和子目录（慎用！）
import shutil

if os.path.exists(work_dir):
    for filename in os.listdir(work_dir):
        file_path = os.path.join(work_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"无法删除 {file_path}: {e}")
else:
    os.makedirs(work_dir)

# 生成psm

In [31]:
pim_file="/home/guci/aiProjects/mda/pim-compiler/examples/blog.md"
llm_config = {
    "llm_model": "google/gemini-2.5-pro",
    "llm_base_url": "https://openrouter.ai/api/v1",
    "llm_api_key_env": "OPENROUTER_API_KEY",
    "llm_temperature": 0
}

# llm_config = {
#     "llm_model": "kimi-k2-turbo-preview",
#     "llm_base_url": "https://api.moonshot.cn/v1",
#     "llm_api_key_env": "MOONSHOT_API_KEY",
#     "llm_temperature": 0
# }

# llm_config = {
#     "llm_model": "deepseek-chat",
#     "llm_base_url": "https://api.deepseek.com/v1",
#     "llm_api_key_env": "DEEPSEEK_API_KEY",
#     "llm_temperature": 0
# }

# llm_config = {
#     "llm_model": "qwen/qwen3-coder",
#     "llm_base_url": "https://openrouter.ai/api/v1",
#     "llm_api_key_env": "OPENROUTER_API_KEY",
#     "llm_temperature": 0,
#     "context_window": 16384*2
# }



config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    knowledge_files=["knowledge/mda/pim_to_psm_knowledge.md"],  # 使用生成专用知识
    enable_project_exploration=False,
    **llm_config
)

psm_generation_agent = GenericReactAgent(config, name="psm_generation_agent")
psm_generation_agent.interface = """psm生成专家 - 专注于快速生成高质量psm(平台特定模型)

# 能力：
- 根据pim生成psm

# 输入:
pim文件路径，psm文件名

# 示例：

## pim文件                                
[pim文件名]

## 任务
根据上面的pim生成psm,文件名是[psm文件名]

## 成功判定条件
生成了完整的psm文件
"""

# x=psm_generation_agent.execute_task(f"""
                                
# # pim文件                                
# {pim_file}

# # 任务
# 根据上面的pim生成psm,文件名是blog_psm.md

# # 成功判定条件
# 1. 生成了完整的psm文件
  
# """)

# if x is None:
#     print("返回结果为空")
# else:
#     print(f"返回结果：{x}")

# 根据psm生成代码

In [32]:


config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    knowledge_files=["knowledge/mda/generation_knowledge.md"],  # 使用生成专用知识
    enable_project_exploration=False,
    **llm_config
)

generation_agent = GenericReactAgent(config, name="generation_agent")
generation_agent.interface = """代码生成专家 - 专注于快速生成高质量代码

# 能力：
- FastAPI应用生成
- 快速交付，不做调试,不运行测试

# 输入:
[psm文件名]

# 返回:
生成的FastAPI应用代码概述

# 示例：

## psm文件                                
[psm文件名]

## 任务
根据上面的psm生成代码

## 成功判定条件
1. 生成了完整的fastapi应用代码
2. 生成了完整的单元测试代码
3. 生成了readme.md
"""

# x=generation_agent.execute_task("""
                                
# # psm文件                                
# blog_psm.md

# # 任务
# 根据上面的psm生成代码

# # 成功判定条件
# 1. 生成了完整的fastapi应用代码
# 2. 生成了完整的单元测试代码
# 3. 生成了readme.md  
# """)

# if x is None:
#     print("返回结果为空")
# else:
#     print(f"返回结果：{x}")

# 修复单元测试

In [33]:
# 检查并压缩调试笔记
check_and_compress_debug_notes(work_dir)

# 获取调试专用工具
debug_tools = create_debug_tools(work_dir)

config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    knowledge_files=[
        "knowledge/mda/debugging_flexible.md",              # 🔄 灵活调试策略
        "knowledge/mda/fix_sqlalchemy_duplicate_table.md",  # SQLAlchemy具体问题修复
        "knowledge/mda/debugging_complete.md",              # 完整调试知识库（合并版）
        "knowledge/mda/debugging_validation.md"             # 验证规范（防止误报成功）
    ],
    enable_project_exploration=False,
    **llm_config
)

debug_agent = GenericReactAgent(config, name="debug_agent", custom_tools=debug_tools)
debug_agent.interface = """修复专家 - 灵活修复错误
    
# 能力：
- 修复fastapi app代码和单元测试，确保100%测试通过

# 示例：
## 任务
修复fastapi app代码和单元测试，让所有测试通过

## 输出
单元测试成功率

## 成功判定条件
单元测试百分之百通过
"""
    
# 简化系统提示，主要依赖知识文件
debug_agent._system_prompt = (debug_agent._system_prompt or "") + """

## 调试任务执行

你是一个灵活的调试Agent。你的知识文件 debugging_flexible.md 包含了：
- 灵活的调试策略
- 成本评估方法
- 实用主义原则
- 多种解决方案

记住：
1. 目标是让测试通过，方法可以灵活
2. 选择最小成本的修复方案
3. 可以修改测试文件或功能文件
4. 持续修复直到所有测试通过
"""

# x=debug_agent.execute_task("""
# # 任务
# 修复单元测试，让所有测试通过

# # 输出
# 单元测试成功率

# # 成功判定条件
# 单元测试百分之百通过
# """)

# print(x)

# 运行app

In [34]:
config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    # knowledge_files=["knowledge/mda/generation_knowledge.md"],  # 使用生成专用知识
    enable_project_exploration=False,
    **llm_config
)

run_app_agent = GenericReactAgent(config, name="run_app_agent")
run_app_agent.interface = """
运行app专家 - 专注于快速运行app

# 能力：
- 运行app

# 示例：

## 任务
1：如果app已经运行，停止app
2：运行app

## 知识
使用curl命令必须加上--noproxy参数
"""

# x=run_app_agent.execute_task("""
# # 任务
# 1：如果app已经运行，停止app
# 2：运行app

# # 知识
# 使用curl命令必须加上--noproxy参数

# """)

# print(x)

# mda工作流

In [ ]:
from core.langchain_agent_tool import AgentToolWrapper, create_langchain_tool
psm_generation_tool=create_langchain_tool(psm_generation_agent)
generation_tool = create_langchain_tool(generation_agent)
debug_agent_tool = create_langchain_tool(debug_agent)
run_app_tool = create_langchain_tool(run_app_agent)

config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    # knowledge_files=["knowledge/mda/coordinator_flexible.md","knowledge/mda/coordinator_workflow.md","knowledge/mda/coordinator_validation.md"],
    enable_project_exploration=False,
    **llm_config  # 使用Gemini配置而不是用户选择的配置
)
    
# 创建主Agent
coordinator_agent = GenericReactAgent(
    config, 
    name='coordinator_agent',
    custom_tools=[psm_generation_tool, generation_tool, debug_agent_tool, run_app_tool]
)

task = f"""
# 目标
从pim文件生成一个完全可工作的FastAPI应用，确保所有测试100%通过，并运行程序

# 输入
- PIM文件：{pim_file}

# 执行流程
1: 生成psm文件
2： 生成fastapi程序
3： 验证单元测试百分之百通过
4： 如果有单元测试失败，则修复单元测试，直到所有测试通过
5： 运行fastapi程序

# 成功判定条件
fastapi app运行成功，所有单元测试通过
"""

x=coordinator_agent.execute_task(task)
print(x)


[coordinator_agent] > Executing task...

👤 用户: 
# 目标
从pim文件生成一个完全可工作的FastAPI应用，确保所有测试100%通过，并运行程序

# 输入
- PIM文件：/home/guci/aiProjects/mda/pim-compiler/examples/blog.md

# 执行流程
1: 生成psm文件
2： 生成fastapi程序
3： 验证单元测试百分之百通过
4： 如果有单元测试失败，则修复单元测试，直到所有测试通过
5： 运行fastapi程序

# 成功判定条件
fastapi app运行成功，所有单元测试通过


🤔 [coordinator_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: psm_generation_agent
   参数: {'task': '根据pim文件 /home/guci/aiProjects/mda/pim-compiler/examples/blog.md 生成psm文件，文件名是 blog_psm.md'}

[psm_generation_agent] > Executing task...

👤 用户: 根据pim文件 /home/guci/aiProjects/mda/pim-compiler/examples/blog.md 生成psm文件，文件名是 blog_psm.md

🤔 [psm_generation_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: read_file
   参数: {'file_path': '/home/guci/aiProjects/mda/pim-compiler/examples/blog.md'}

💬 工具结果 (read_file):
   # 博客系统

## 业务描述
一个简单的博客发布系统，支持文章发布、分类和评论功能。

## 业务实体

### 文章 (Article)
博客的核心内容。

属性：
- 标题：文章的标题
- 内容：文章正文（支持 Markdown）
- 摘要：文章简介
- 作者：发布者名称
- 发布时间：文章发布的时间
- 更新时间：最后修改时间
- 状态：草稿/已发布
- 浏览量：文章被查看的次数

### 分类 (Category)


🚨 [记忆更新] 发现错误纠正：```markdown


🚨 [记忆更新] 发现错误纠正：# 知识库



# 实验

In [25]:
x=run_app_agent.execute_task("""
# 任务
运行app

# 知识
使用curl命令必须加上--noproxy参数

# 成功判定条件
verify it responds to /api/articles/

# 终止条件
Once the app returns data, the task is complete

""")

print(x)


[run_app_agent] > Executing task...

👤 用户: 
# 任务
运行app

# 知识
使用curl命令必须加上--noproxy参数

# 成功判定条件
verify it responds to /api/articles/

# 终止条件
Once the app returns data, the task is complete



🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'uvicorn app.main:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &'}

💬 工具结果 (execute_command):
   Exit code: 0


🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'curl --noproxy 127.0.0.1 http://127.0.0.1:8000/api/articles/'}

💬 工具结果 (execute_command):
   Exit code: 0
[{"title":"string","content":"string","summary":"string","author":"string","status":"draft","category_id":0,"id":1,"published_at":null,"updated_at":"2025-08-10T16:12:13","view_count":0,"category":null},{"title":"aaaaa","content":"string","summary":"string","author":"string","status":"draft","category_id":1,"id":2,"published_at":null,"updated_at":"2025-08-10T16:13:20","view_count":0,"category":null}]
Errors:
  % Total    


🚨 [记忆更新] 发现错误纠正：# 知识库



# 终止程序

In [ ]:
x=run_app_agent.execute_task("""
停止app

# 终止条件
8000端口没有进程

# 知识
当任务完成后，明确说明完成状态并停止执行。
不要重复验证已经成功的操作。


""")

print(x)


[run_app_agent] > Executing task...

👤 用户: 
停止app

# 终止条件
8000端口没有进程

# 知识
当任务完成后，明确说明完成状态并停止执行。
不要重复验证已经成功的操作。




🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'lsof -t -i:8000'}

💬 工具结果 (execute_command):
   Exit code: 0
1085979


🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'kill 1085979'}

💬 工具结果 (execute_command):
   Exit code: 0


🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'lsof -t -i:8000'}

💬 工具结果 (execute_command):
   Exit code: 1


🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'lsof -t -i:8000'}

🤔 [run_app_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'lsof -t -i:8000'}

💬 工具结果 (None):
   没有找到在8000端口上运行的进程。任务已完成。
没有在8000端口上运行的进程。任务已完成。

[run_app_agent] > Task completed.

=== [run_app_agent] 最终结果 ===

没有找到在8000端口上运行的进程。任务已完成。
没有在8000端口上运行的进程。任务已完成。
没有找到在8000端口上运行的进程。任务已完成。
没有在8000端口上运行的进程。任务已完成。



🚨 [记忆更新] 发现错误纠正：# 知识库



# 验证单元测试

In [13]:
config = ReactAgentConfig(
    work_dir=work_dir,
    memory_level=MemoryLevel.SMART,
    # knowledge_files=["knowledge/mda/generation_knowledge.md"],  # 使用生成专用知识
    enable_project_exploration=False,
    **llm_config
)

mda_agent = GenericReactAgent(config, name="mda_agent")

x=mda_agent.execute_task("""
# 任务
验证单元测试成功率

# 输出
单元测试成功率

""")

print(x)


[mda_agent] > Executing task...

👤 用户: 
# 任务
验证单元测试成功率

# 输出
单元测试成功率



🤔 [mda_agent] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'pytest'}

💬 工具结果 (execute_command):
   Exit code: 0
============================= test session starts ==============================
platform linux -- Python 3.10.12, pytest-7.4.3, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
PyQt5 5.15.9 -- Qt runtime 5.15.2 -- Qt compiled 5.15.2
benchmark: 5.1.0 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /home/guci/aiProjects/mda/pim-compiler
configfile: ...
   [省略 621 字符]

💬 工具结果 (None):
   单元测试成功率: 100% (3/3 passed)

[mda_agent] > Task completed.

=== [mda_agent] 最终结果 ===

单元测试成功率: 100% (3/3 passed)
单元测试成功率: 100% (3/3 passed)



🚨 [记忆更新] 发现错误纠正：# 知识库

